In [58]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix
#### 2-qubit pauli basis
pauli_basis=[np.asarray(np.kron(i,j)) for i in [I, X, Y, Z] for j in [I, X, Y, Z]]
#### 2-qubit |0> and |1>
e_state=np.kron(np.asarray([[0,0],[0,1]]),np.asarray([[0,0],[0,1]]))
g_state=np.kron(np.asarray([[1,0],[0,0]]),np.asarray([[1,0],[0,0]]))
####Density matrix to PTM
def densityop_01_2_pauli(density_op):
    #assert density_op.ndim !=4,'wrong dim'
    A=np.asarray([np.trace(density_op@i_den) for i_den in pauli_basis]).reshape(density_op.size,1)
    return A
def densityop_pauli_2_01(density_op):
    #assert density_op.shape == (len(density_op),1),'wrong dim'
    A_den=np.asarray([density_op[i_den]*pauli_basis[i_den] for i_den in range(16)])
    B_den=np.zeros((4,4))
    for i_den in A_den:
        B_den=B_den+i_den
    return B_den/4
def dephaseing_kraus(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0,3]:
        A[i_deph] = A[i_deph]*(1-p)
    for i_deph in [1,2]:
        A[i_deph] = A[i_deph]*np.sqrt(p*(1-p))
    return A
def depolarizing_kraus(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-p)
    for i_depo in range(1,len(A)):
        A[i_depo]=A[i_depo]*np.sqrt(p/15)
    return A
if __name__ == '__main__':

####### input
    rb_types=int(input('-1 rb, 0 decomp_rb, 1 prb\n'))
    max_seq= int(input('max seq \n'))
####### rb gate number i    
    for i in tqdm(sequence_length,ascii=True,desc="sequence_length",dynamic_ncols=True):
####### generate rb sequence        
        pool = Pool()
        seq = pool.map(sequence_list_length_by_1,[i]*types)
        pool.close()
        pool.join()
####### rb sequence into probablility
        types_probability=[]
        m_average=[]
        pool = mp.Pool()
        repeat_probability = pool.starmap(sequence_to_probability,[(i,None,d,e) for d in range(1) for e in seq ])
        pool.close()
        pool.join()

1.0  .   .   .  
 .   .   .   .  
 .   .   .   .  
 .   .   .   .  


'####### input\n    rb_types=int(input(\'-1 rb, 0 decomp_rb, 1 prb\n\'))\n    max_seq= int(input(\'max seq \n\'))\n####### rb gate number i    \n    for i in tqdm(sequence_length,ascii=True,desc="sequence_length",dynamic_ncols=True):\n####### generate rb sequence        \n        pool = Pool()\n        seq = pool.map(sequence_list_length_by_1,[i]*types)\n        pool.close()\n        pool.join()\n####### rb sequence into probablility\n        types_probability=[]\n        m_average=[]\n        pool = mp.Pool()\n        repeat_probability = pool.starmap(sequence_to_probability,[(i,None,d,e) for d in range(1) for e in seq ])\n        pool.close()\n        pool.join()'

In [1]:
#rb seq w/inverse
import random
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import tqdm
from multiprocessing import Process, Pool
#### 2-qubit pauli basis
with open('Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
def rb_seq_fun(seq_number):
    np.random.seed()
    rb_seq_index= np.random.randint(low=0, high=11520, size=seq_number).tolist()
    rb_seq_kraus=np.eye(4)
    for i_rb in rb_seq_index:
        rb_seq_kraus=Cliff_kruas[i_rb]@rb_seq_kraus
    for i_rb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_rb]@np.conj(rb_seq_kraus).T)),4)):
            rb_seq_index.append(i_rb)
            break
        else:
            assert i_rb != 11519, 'no inverse'
            continue
    return rb_seq_index
if __name__ == '__main__':
    rb_seq={}
    Max=int(input('max sequence number'))
    types=int(input('different types sequence '))
    for i in range(1,Max):

        pool=Pool()
        _rb_seq=pool.map(rb_seq_fun,[i]*types)
        rb_seq[i]=_rb_seq
        pool.close()
        pool.join()
    with open('rb_seq.pkl', 'wb') as f:
        pickle.dump(rb_seq, f)

In [13]:
#prb seq w/inverse
import random
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import tqdm
from multiprocessing import Process, Pool
#### 2-qubit pauli basis
with open('H_P_CNOT_kruas.pkl', 'rb') as f:
    Primitive_kruas = pickle.load(f)
#### weights ?
Primitive_weights=[]

def prb_seq_fun(seq_number):
    np.random.seed()
    prb_seq_index= random.choices([i for i in range(len(Primitive_kruas))], weights=Primitive_weights, k=seq_number).tolist()
    prb_seq_kraus=np.eye(4)
    for i_rb in prb_seq_index:
        prb_seq_kraus=Primitive_kruas[i_rb]@prb_seq_kraus
    for i_rb in range(len(Primitive_kruas)):
        if (np.isclose(abs(np.trace(Primitive_kruas[i_rb]@np.conj(prb_seq_kraus).T)),4)):
            prb_seq_index.append(i_rb)
            break
        else:
            assert i_rb != len(Primitive_kruas)-1, 'no inverse'
            continue
    return prb_seq_index
if __name__ == '__main__':
    prb_seq={}
    Max=int(input('max sequence number'))
    types=int(input('different types sequence '))
    for i in range(1,Max):

        pool=Pool()
        _prb_seq=pool.map(prb_seq_fun,[i]*types)
        prb_seq[i]=_prb_seq
        pool.close()
        pool.join()
    with open('prb_seq.pkl', 'wb') as f:
        pickle.dump(prb_seq, f)

FileNotFoundError: [Errno 2] No such file or directory: 'Primitive_kruas.pkl'